In [3]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns

ruta = os.path.abspath('../Datasets')

In [4]:
df_compra = pd.read_csv(ruta + '/Compra.csv', sep=',', encoding='utf-8')

In [5]:
#eliminando IdCompra
df_compra.drop(['IdCompra'],axis=1,inplace=True)
#filas duplicadas
df_compra.duplicated().sum()

0

## Detección de Outliers

In [6]:
Q1 = df_compra['Precio'].quantile(.25)
Q3 = df_compra['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [7]:
ubicacion_outliers = (df_compra.Precio < minimo) | (df_compra.Precio > maximo)
df_compra[ubicacion_outliers].sort_values('Precio')

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
580,4/24/2015,2015,4,201504,42782,10,2691.85,7
5833,4/27/2018,2018,4,201804,42936,3,2692.87,14
8607,10/25/2019,2019,1,201901,42782,22,2693.43,2
10422,7/31/2020,2020,7,202007,42780,6,2694.01,6
730,5/29/2015,2015,5,201505,42780,12,2694.41,6
...,...,...,...,...,...,...,...,...
6805,10/26/2018,2018,1,201801,42773,2,577223.00,13
7414,2/22/2019,2019,2,201902,42773,13,580886.00,6
8763,11/29/2019,2019,1,201901,42779,17,688826.00,10
11272,11/27/2020,2020,11,202011,42779,7,707004.00,8


In [10]:
#analizando uno de los posibles outliers
df_compra[df_compra.IdProducto == 42782].head()

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
130,1/30/2015,2015,1,201501,42782,6,2678.34,1
580,4/24/2015,2015,4,201504,42782,10,2691.85,7
732,5/29/2015,2015,5,201505,42782,3,2347.42,13
878,6/26/2015,2015,6,201506,42782,4,2422.17,3
1186,8/28/2015,2015,8,201508,42782,4,2031.08,13


Podemos observar que el supuesto valor que es un outlier para la columna Precio, es un valor normal para el tipo de producto: 42782.
Lo que nos hace concluir que la estrategia que estamos usando para detectar outliers no es la adecuada.
Ya que la empresa tiene diferentes tipos de productos, buscaremos los valores atipicos en función del IdProducto.

In [12]:
#numero de diferentes productos existentes en la cartera de la empresa
df_compra.IdProducto.nunique()

283

In [13]:
#rango intercuantílico para un solo producto
df_id = df_compra[df_compra.IdProducto == 42782]
Q1 = df_id['Precio'].quantile(.25)
Q3 = df_id['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [16]:
ubicacion_outliers = (df_id.Precio < minimo) | (df_id.Precio > maximo)
df_id[ubicacion_outliers]

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
9327,2/28/2020,2020,2,202002,42782,12,214566.0,4


Vemos que la estrategia de busqueda de outliers por IdProducto es más certero. Generalicemos el código para todo los productos.